In [1]:
import sys, os, glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii, fits
from astropy import table
from astropy.wcs import WCS
from astropy.convolution import convolve_fft
from scipy.interpolate import RegularGridInterpolator
import webbpsf
from jwst_kernels.make_kernels import make_jwst_cross_kernel, make_jwst_kernel_to_Gauss,plot_kernel
from jwst_kernels.evaluate_kernels import find_safe_kernel, plot_evaluate
from jwst_kernels.make_psf import save_miri_PSF
from reproject import reproject_interp, reproject_adaptive, reproject_exact

In [2]:
input_filter = {'camera':'MIRI', 'filter':'F770W'}
hdulist = fits.open("./DataFiles/M82_IR80_Spitzer.fits")
head = hdulist[0].header
print(head)
grid = WCS(head)


SIMPLE  =                    T /image conforms to FITS standard                 BITPIX  =                  -32 /bits per data value                             NAXIS   =                    2 /number of axes                                  NAXIS1  =                 1253 /                                                NAXIS2  =                 1258 /                                                EXTEND  =                    T /file may contain extensions                     ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 2003' / FITS file originator        DATE    = '2007-04-05T05:08:47' / Date FITS file was generated                  IRAF-TLM= '01:08:59 (05/04/2007)' / Time of last modification                   OBJECT  = 'NGC3034 '           / Name of the object observed                    CREATOR = 'S14.0.0 '           / SW version used to create this FITS file       TELESCOP= 'Spitzer '           / SPITZER Space Telescope                        INSTRUME= 'IRAC    '           / SPITZER

In [11]:
#save_miri_PSF(["F770W"], output_dir="./")

building PSF F1130W


In [5]:
target_gaussian = {'fwhm':1.98} #Input FWHM of target resolution here
kk = make_jwst_kernel_to_Gauss(input_filter, target_gaussian,psf_dir="./", outdir="./", detector_effects=True) #Saves kernel as fits file

In [3]:
#Copied and pasted from github
def get_pixscale(hdu):
    """Get pixel scale from header.

    Checks HDU header and returns a pixel scale

    Args:
        hdu: hdu to get pixel scale for
    """

    for pixel_keyword in ["XPIXSIZE", "CDELT1", "CD1_1", "PIXELSCL"]:
        try:
            try:
                pix_scale = np.abs(float(hdu.header[pixel_keyword]))
            except ValueError:
                continue
            if pixel_keyword in ["CDELT1", "CD1_1"]:
                pix_scale = WCS(hdu.header).proj_plane_pixel_scales()[0].value * 3600
                # pix_scale *= 3600
            return pix_scale
        except KeyError:
            pass

    raise Warning("No pixel scale found")

def do_jwst_convolution(
        file_in,
        file_out,
        file_kernel,
        blank_zeros=True,
        output_grid=None,
        reproject_func="interp",
):
    """
    Convolves input image with an input kernel, and writes to disk.

    Will also process errors and do reprojection, if specified

    Args:
        file_in: Path to image file
        file_out: Path to output file
        file_kernel: Path to kernel for convolution
        blank_zeros: If True, then all zero values will be set to NaNs. Defaults to True
        output_grid: None (no reprojection to be done) or tuple (wcs, shape) defining the grid for reprojection.
            Defaults to None
        reproject_func: Which reproject function to use. Defaults to 'interp',
            but can also be 'exact' or 'adaptive'
    """
    if reproject_func == "interp":
        r_func = reproject_interp
    elif reproject_func == "exact":
        r_func = reproject_exact
    elif reproject_func == "adaptive":
        r_func = reproject_adaptive
    else:
        raise ValueError(f"look at github stupid")
    with fits.open(file_kernel) as kernel_hdu:
        kernel_pix_scale = get_pixscale(kernel_hdu[0])
        # Note the shape and grid of the kernel as input
        kernel_data = kernel_hdu[0].data
        kernel_hdu_length = kernel_hdu[0].data.shape[0]
        original_central_pixel = (kernel_hdu_length - 1) / 2
        original_grid = (
                                np.arange(kernel_hdu_length) - original_central_pixel
                        ) * kernel_pix_scale
    with fits.open(file_in) as image_hdu:
        if blank_zeros:
            # make sure that all zero values were set to NaNs, which
            # astropy convolution handles with interpolation
            image_hdu["ERR"].data[(image_hdu["SCI"].data == 0)] = np.nan
            image_hdu["SCI"].data[(image_hdu["SCI"].data == 0)] = np.nan

        image_pix_scale = get_pixscale(image_hdu["SCI"])

        # Calculate kernel size after interpolating to the image pixel
        # scale. Because sometimes there's a little pixel scale rounding
        # error, subtract a little bit off the optimum size (Tom
        # Williams).

        interpolate_kernel_size = (
                np.floor(kernel_hdu_length * kernel_pix_scale / image_pix_scale) - 2
        )

        # Ensure the kernel has a central pixel

        if interpolate_kernel_size % 2 == 0:
            interpolate_kernel_size -= 1
        # Define a new coordinate grid onto which to project the kernel
        # but using the pixel scale of the image

        new_central_pixel = (interpolate_kernel_size - 1) / 2
        new_grid = (
            np.arange(interpolate_kernel_size) - new_central_pixel
                   ) * image_pix_scale
        x_coords_new, y_coords_new = np.meshgrid(new_grid, new_grid)

        # Do the reprojection from the original kernel grid onto the new
        # grid with pixel scale matched to the image

        grid_interpolated = RegularGridInterpolator(
            (original_grid, original_grid),
            kernel_data,
            bounds_error=False,
            fill_value=0.0,
        )
        kernel_interp = grid_interpolated(
            (x_coords_new.flatten(), y_coords_new.flatten())
        )
        kernel_interp = kernel_interp.reshape(x_coords_new.shape)

        # Ensure the interpolated kernel is normalized to 1
        kernel_interp = kernel_interp / np.nansum(kernel_interp)
        # Now with the kernel centered and matched in pixel scale to the
        # input image use the FFT convolution routine from astropy to
        # convolve.

        conv_im = convolve_fft(
            image_hdu["SCI"].data,
            kernel_interp,
            allow_huge=True,
            preserve_nan=True,
            fill_value=np.nan,
        )

        # Convolve errors (with kernel**2, do not normalize it).
        # This, however, doesn't account for covariance between pixels
        conv_err = np.sqrt(
            convolve_fft(
                image_hdu["ERR"].data ** 2,
                kernel_interp ** 2,
                preserve_nan=True,
                allow_huge=True,
                normalize_kernel=False,
            )
        )

        image_hdu["SCI"].data = conv_im
        image_hdu["ERR"].data = conv_err
        
        if output_grid is None:
            image_hdu.writeto(file_out, overwrite=True)
        else:
            # Reprojection to target wcs grid define in output_grid
            target_wcs, target_shape = output_grid
            hdulist_out = fits.HDUList([fits.PrimaryHDU(header=image_hdu[0].header)])

            repr_data, fp = r_func(
                (conv_im, image_hdu["SCI"].header),
                output_projection=target_wcs,
                shape_out=target_shape,
            )
            fp = fp.astype(bool)
            repr_data[~fp] = np.nan
            header = image_hdu["SCI"].header
            header.update(target_wcs.to_header())
            hdulist_out.append(fits.ImageHDU(data=repr_data, header=header, name="SCI"))

            # Note - this ignores the errors of interpolation and thus the resulting errors might be underestimated
            repr_err = r_func(
                (conv_err, image_hdu["SCI"].header),
                output_projection=target_wcs,
                shape_out=target_shape,
                return_footprint=False,
            )
            repr_err[~fp] = np.nan
            header = image_hdu["ERR"].header
            hdulist_out.append(fits.ImageHDU(data=repr_err, header=header, name="ERR"))

            hdulist_out.writeto(file_out, overwrite=True)

In [4]:
do_jwst_convolution(file_in="./DataFiles/JWST_770w_IR_data.fits", file_out="./JWSTConvToSpitzer.fits", file_kernel="./f770w_to_gauss1p98.fits", output_grid=(grid,(1000,1000)))

Set DATE-AVG to '2023-12-31T19:43:39.963' from MJD-AVG.
Set DATE-END to '2024-01-01T01:32:08.328' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to     8.864333 from OBSGEO-[XYZ].
Set OBSGEO-H to 1694877744.107 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


INFO: 
        Inconsistent SIP distortion information is present in the current WCS:
        SIP coefficients were detected, but CTYPE is missing "-SIP" suffix,
        therefore the current WCS is internally inconsistent.

        Because relax has been set to True, the resulting output WCS will have
        "-SIP" appended to CTYPE in order to make the header internally consistent.

        However, this may produce incorrect astrometry in the output WCS, if
        in fact the current WCS is already distortion-corrected.

        Therefore, if current WCS is already distortion-corrected (eg, drizzled)
        then SIP distortion components should not apply. In that case, for a WCS
        that is already distortion-corrected, please remove the SIP coefficients
        from the header.

         [astropy.wcs.wcs]
